In [ ]:

#importing libraries
import pandas as pd
import seaborn as sns
import string 
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import warnings

warnings.filterwarnings("ignore")
nltk.download('stopwords')
nltk.download('wordnet')
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VIKRAM\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\VIKRAM\AppData\Roaming\nltk_data...
True
#reading the dataset 
#dataset: https://www.kaggle.com/uciml/sms-spam-collection-data
msg=pd.read_csv('C:/New folder/spam_detector.csv',encoding='latin-1')
msg.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)
msg.rename(columns={'v1':'label','v2':'text'},inplace=True)
msg.head()
label	text
0	ham	Go until jurong point, crazy.. Available only ...
1	ham	Ok lar... Joking wif u oni...
2	spam	Free entry in 2 a wkly comp to win FA Cup fina...
3	ham	U dun say so early hor... U c already then say...
4	ham	Nah I don't think he goes to usf, he lives aro...
#mapping ham=0 and spam=1
for i in msg.index:
  if msg['label'][i]=='ham':
    msg['label'][i]=0
  else:
    msg['label'][i]=1
msg.head()
label	text
0	0	Go until jurong point, crazy.. Available only ...
1	0	Ok lar... Joking wif u oni...
2	1	Free entry in 2 a wkly comp to win FA Cup fina...
3	0	U dun say so early hor... U c already then say...
4	0	Nah I don't think he goes to usf, he lives aro...
#category count plot (count of spam and ham)
sns.countplot(msg.label)
<Axes: ylabel='count'>

#data description grouped by labels 
msg.groupby('label').describe()
text
count	unique	top	freq
label				
0	4825	4516	Sorry, I'll call later	30
1	747	653	Please call our customer service representativ...	4
#dropping duplicate rows
msg=msg.drop_duplicates()
msg.groupby('label').describe()
text
count	unique	top	freq
label				
0	4516	4516	Go until jurong point, crazy.. Available only ...	1
1	653	653	Free entry in 2 a wkly comp to win FA Cup fina...	1
#adding length column to the dataset 
msg['length']=msg['text'].apply(len)
msg.head()
label	text	length
0	0	Go until jurong point, crazy.. Available only ...	111
1	0	Ok lar... Joking wif u oni...	29
2	1	Free entry in 2 a wkly comp to win FA Cup fina...	155
3	0	U dun say so early hor... U c already then say...	49
4	0	Nah I don't think he goes to usf, he lives aro...	61
msg[msg.label==0].describe()
length
count	4516.000000
mean	70.459256
std	56.358207
min	2.000000
25%	34.000000
50%	52.000000
75%	90.000000
max	910.000000
sns.distplot(a=msg[msg['label']==0].length,kde=False)
<Axes: xlabel='length'>

msg[msg.label==1].describe()
length
count	653.000000
mean	137.891271
std	30.137753
min	13.000000
25%	132.000000
50%	149.000000
75%	157.000000
max	224.000000
sns.distplot(a=msg[msg['label']==1].length,kde=False)
<Axes: xlabel='length'>

#examining spam texts
for i in range(50):
  if msg['label'][i]==1:
    print(msg['text'][i])
Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv
WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030
SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info
URGENT! You have won a 1 week FREE membership in our å£100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18
XXXMobileMovieClub: To use your credit, click the WAP link in the next txt message or click here>> http://wap. xxxmobilemovieclub.com?n=QJKGIGHJJGCBL
England v Macedonia - dont miss the goals/team news. Txt ur national team to 87077 eg ENGLAND to 87077 Try:WALES, SCOTLAND 4txt/Ì¼1.20 POBOXox36504W45WQ 16+
Thanks for your subscription to Ringtone UK your mobile will be charged å£5/month Please confirm by replying YES or NO. If you reply NO you will not be charged
07732584351 - Rodger Burns - MSG = We tried to call you re your reply to our sms for a free nokia mobile + free camcorder. Please call now 08000930705 for delivery tomorrow
msg['contain']=msg['text'].str.contains('£').map({False:0,True:1})
msg['contain']=msg['contain']|msg['text'].str.contains('%').map({False:0,True:1})
msg['contain']=msg['contain']|msg['text'].str.contains('€').map({False:0,True:1})
msg['contain']=msg['contain']|msg['text'].str.contains('\$').map({False:0,True:1})
msg['contain']=msg['contain']|msg['text'].str.contains("T&C").map({False:0,True:1})
msg['contain']=msg['contain']|msg['text'].str.contains("www|WWW").map({False:0,True:1})
msg['contain']=msg['contain']|msg['text'].str.contains("http|HTTP").map({False:0,True:1})
msg['contain']=msg['contain']|msg['text'].str.contains("https|HTTPS").map({False:0,True:1})
msg['contain']=msg['contain']|msg['text'].str.contains("@").map({False:0,True:1})
msg['contain']=msg['contain']|msg['text'].str.contains("email|Email|EMAIL").map({False:0,True:1})
msg['contain']=msg['contain']|msg['text'].str.contains("SMS|sms|FREEPHONE").map({False:0,True:1})
msg['contain']=msg['contain']|msg['text'].str.contains("\d{11}",regex=True).map({False:0,True:1})
msg['contain']=msg['contain']|msg['text'].str.contains("\d{10}",regex=True).map({False:0,True:1})
msg['contain']=msg['contain']|msg['text'].str.contains("\d{5}",regex=True).map({False:0,True:1})

msg.head()
label	text	length	contain
0	0	Go until jurong point, crazy.. Available only ...	111	0
1	0	Ok lar... Joking wif u oni...	29	0
2	1	Free entry in 2 a wkly comp to win FA Cup fina...	155	1
3	0	U dun say so early hor... U c already then say...	49	0
4	0	Nah I don't think he goes to usf, he lives aro...	61	0
sns.distplot(a=msg[msg['label']==0].contain,kde=False)
<Axes: xlabel='contain'>

sns.distplot(a=msg[msg['label']==1].contain,kde=False)
<Axes: xlabel='contain'>

#data cleaning/preprocessing - removing punctuation and digits 
msg['cleaned_text']=""

for i in msg.index:
  updated_list=[]
  for j in range(len(msg['text'][i])):
    if msg['text'][i][j] not in string.punctuation:
      if msg['text'][i][j].isdigit()==False:
        updated_list.append(msg['text'][i][j])
  updated_string="".join(updated_list)
  msg['cleaned_text'][i]=updated_string

msg.drop(['text'],axis=1,inplace=True)
msg.head() 
label	length	contain	cleaned_text
0	0	111	0	Go until jurong point crazy Available only in ...
1	0	29	0	Ok lar Joking wif u oni
2	1	155	1	Free entry in a wkly comp to win FA Cup final...
3	0	49	0	U dun say so early hor U c already then say
4	0	61	0	Nah I dont think he goes to usf he lives aroun...
#data cleaning/preprocessing - tokenization and convert to lower case 
msg['token']=""

for i in msg.index:
  msg['token'][i]=re.split("\W+",msg['cleaned_text'][i].lower())

msg.head()
label	length	contain	cleaned_text	token
0	0	111	0	Go until jurong point crazy Available only in ...	[go, until, jurong, point, crazy, available, o...
1	0	29	0	Ok lar Joking wif u oni	[ok, lar, joking, wif, u, oni]
2	1	155	1	Free entry in a wkly comp to win FA Cup final...	[free, entry, in, a, wkly, comp, to, win, fa, ...
3	0	49	0	U dun say so early hor U c already then say	[u, dun, say, so, early, hor, u, c, already, t...
4	0	61	0	Nah I dont think he goes to usf he lives aroun...	[nah, i, dont, think, he, goes, to, usf, he, l...
#data cleaning/preprocessing - stopwords
msg['updated_token']=""
stopwords=nltk.corpus.stopwords.words('english')

for i in msg.index:
  updated_list=[]
  for j in range(len(msg['token'][i])):
    if msg['token'][i][j] not in stopwords:
      updated_list.append(msg['token'][i][j])
  msg['updated_token'][i]=updated_list

msg.drop(['token'],axis=1,inplace=True)
msg.head()
label	length	contain	cleaned_text	updated_token
0	0	111	0	Go until jurong point crazy Available only in ...	[go, jurong, point, crazy, available, bugis, n...
1	0	29	0	Ok lar Joking wif u oni	[ok, lar, joking, wif, u, oni]
2	1	155	1	Free entry in a wkly comp to win FA Cup final...	[free, entry, wkly, comp, win, fa, cup, final,...
3	0	49	0	U dun say so early hor U c already then say	[u, dun, say, early, hor, u, c, already, say]
4	0	61	0	Nah I dont think he goes to usf he lives aroun...	[nah, dont, think, goes, usf, lives, around, t...
#data cleaning/preprocessing - lemmatization 
msg['lem_text']=""
wordlem=nltk.WordNetLemmatizer()

for i in msg.index:
  updated_list=[]
  for j in range(len(msg['updated_token'][i])):
    updated_list.append(wordlem.lemmatize(msg['updated_token'][i][j]))
  msg['lem_text'][i]=updated_list 

msg.drop(['updated_token'],axis=1,inplace=True)
msg.head()
label	length	contain	cleaned_text	lem_text
0	0	111	0	Go until jurong point crazy Available only in ...	[go, jurong, point, crazy, available, bugis, n...
1	0	29	0	Ok lar Joking wif u oni	[ok, lar, joking, wif, u, oni]
2	1	155	1	Free entry in a wkly comp to win FA Cup final...	[free, entry, wkly, comp, win, fa, cup, final,...
3	0	49	0	U dun say so early hor U c already then say	[u, dun, say, early, hor, u, c, already, say]
4	0	61	0	Nah I dont think he goes to usf he lives aroun...	[nah, dont, think, go, usf, life, around, though]
#data cleaning/preprocessing - merging token
msg['final_text']=""

for i in msg.index:
  updated_string=" ".join(msg['lem_text'][i])
  msg['final_text'][i]=updated_string

msg.drop(['cleaned_text','lem_text'],axis=1,inplace=True)
msg.head()
label	length	contain	final_text
0	0	111	0	go jurong point crazy available bugis n great ...
1	0	29	0	ok lar joking wif u oni
2	1	155	1	free entry wkly comp win fa cup final tkts st ...
3	0	49	0	u dun say early hor u c already say
4	0	61	0	nah dont think go usf life around though
#separating target and features
y=pd.DataFrame(msg.label)
x=msg.drop(['label'],axis=1)
#splitting the data (80:20 ratio)
x_train,x_val,y_train,y_val=train_test_split(x,y,train_size=0.8,test_size=0.2,random_state=0)
#count vectorization 
cv=CountVectorizer(max_features=5000)
temp_train=cv.fit_transform(x_train['final_text']).toarray()
temp_val=cv.transform(x_val['final_text']).toarray()
#tfidf
tf=TfidfTransformer()
temp_train=tf.fit_transform(temp_train)
temp_val=tf.transform(temp_val)
#merging temp datafram with original dataframe
temp_train=pd.DataFrame(temp_train.toarray(),index=x_train.index)
temp_val=pd.DataFrame(temp_val.toarray(),index=x_val.index)
x_train=pd.concat([x_train,temp_train],axis=1,sort=False)
x_val=pd.concat([x_val,temp_val],axis=1,sort=False)

x_train.head()
length	contain	final_text	0	1	2	3	4	5	6	...	4990	4991	4992	4993	4994	4995	4996	4997	4998	4999
3794	45	0	also remember bead dont come ever	0.0	0.0	0.0	0.0	0.0	0.0	0.0	...	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
4290	25	0	call ure done	0.0	0.0	0.0	0.0	0.0	0.0	0.0	...	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
2603	99	0	im arestaurant eating squid wanna dosomething ...	0.0	0.0	0.0	0.0	0.0	0.0	0.0	...	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
3452	274	0	nowadays people notixiquating laxinorficated o...	0.0	0.0	0.0	0.0	0.0	0.0	0.0	...	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
3132	25	0	message food	0.0	0.0	0.0	0.0	0.0	0.0	0.0	...	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
5 rows × 5003 columns

#dropping the final_text column
x_train.drop(['final_text'],axis=1,inplace=True)
x_val.drop(['final_text'],axis=1,inplace=True)

x_train.head()
length	contain	0	1	2	3	4	5	6	7	...	4990	4991	4992	4993	4994	4995	4996	4997	4998	4999
3794	45	0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	...	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
4290	25	0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	...	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
2603	99	0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	...	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
3452	274	0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	...	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
3132	25	0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	...	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
5 rows × 5002 columns

#converting the labels to int datatype (for model training)
y_train=y_train.astype(int)
y_val=y_val.astype(int)
#Multinomial Naive Bayes
model=MultinomialNB()
(x_train,y_train)

print("Multinomial Naive Bayes:",accuracy_score)
Multinomial Naive Bayes: <function accuracy_score at 0x00000230EF1544A0>
#Decision Tree
model=DecisionTreeClassifier(random_state=0)
(x_train,y_train)

print("Decision Tree:",accuracy_score)
Decision Tree: <function accuracy_score at 0x00000230EF1544A0>
#Random Forest
model=RandomForestClassifier(n_estimators=100,random_state=0)


print("Random Forest:",accuracy_score)
Random Forest: <function accuracy_score at 0x00000230EF1544A0>